In [1]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [3]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [4]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [5]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,BCE00606125,2025.0,32,26.686976,20564.31,548799.25,0.0033,0.0408,7.585019e+11,1382111.72,0.01,0.000000,1382111.72,0.0,0.02,AAA,0.999
1,MOSTRADOR COTA,BCE00606125,2025.0,33,21.349581,20564.31,439039.40,0.0027,0.0639,1.800062e+05,0.41,0.01,5.000000,1382112.13,1.0,0.24,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,42,21.198134,51663.09,1095161.10,0.0067,0.0067,5.256773e+05,0.48,0.01,7.216495,1382112.61,1.0,0.23,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,43,19.078321,51663.09,985645.01,0.0060,0.0127,3.942580e+05,0.40,0.02,6.073620,1382113.01,1.0,0.23,AAA,0.999
4,MOSTRADOR COTA,BCE00606125,2025.0,34,17.079665,20564.31,351231.53,0.0021,0.0778,1.791281e+05,0.51,0.02,6.000000,1382113.52,1.0,0.25,AAA,0.999
5,MOSTRADOR COTA,DCS10536137,2025.0,31,16.499440,51663.09,852412.05,0.0052,0.0179,1.883831e+06,2.21,0.02,3.000000,1382115.73,0.8,0.20,AAA,0.999
6,MOSTRADOR COTA,DAB02570025,2025.0,35,15.514379,13771.02,213648.82,0.0013,0.1156,3.204730e+04,0.15,0.03,1.849057,1382115.88,0.8,0.23,AAA,0.999
7,MOSTRADOR COTA,DAB14570025,2025.0,35,15.282377,10117.70,154622.51,0.0009,0.1674,2.937830e+04,0.19,0.03,3.169811,1382116.07,1.0,0.29,AAA,0.999
8,MOSTRADOR COTA,DCS10536137,2025.0,32,14.849496,51663.09,767170.85,0.0047,0.0226,2.608381e+05,0.34,0.03,3.921569,1382116.41,1.0,0.24,AAA,0.999
9,MOSTRADOR COTA,DRGCOOLR035,2025.0,42,14.776802,13412.99,198201.10,0.0012,0.1231,2.093999e+11,1056502.35,0.04,0.000000,2438618.76,0.0,0.07,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [6]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [7]:
df_std[(df_std["semana"]==42) & (df_std["store_name"]=="SUCURSAL BARRANQUILLA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
54022,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,42,45.108876,10117.70,456398.07,0.0005,0.2210,104971.6,0.23,0.01,3.602190,1.89,1.0,0.31,AAA,0.999,71.89,477.87,387.66
54023,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,42,44.945801,13771.02,618949.52,0.0007,0.1456,129979.4,0.21,0.01,4.030628,2.10,1.0,0.29,AAA,0.999,80.31,484.82,394.93
54060,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,42,30.295668,20581.34,623525.44,0.0007,0.1435,87293.6,0.14,0.06,3.039024,9.25,1.0,0.31,AAA,0.999,36.83,309.49,248.90
54073,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,42,28.605627,31057.73,888425.84,0.0010,0.0947,124379.6,0.14,0.08,2.131343,10.74,0.8,0.26,AAA,0.999,35.60,293.05,235.84
54079,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,42,27.100836,10883.85,294961.43,0.0003,0.3363,58992.3,0.20,0.08,3.859649,11.71,1.0,0.39,AAA,0.999,49.78,293.69,239.48
54089,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,42,25.983102,32494.89,844318.04,0.0009,0.0966,92875.0,0.11,0.09,2.259091,2045928.04,0.8,0.28,AAA,0.999,31.52,265.37,213.40
54118,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,42,22.553109,11371.65,256466.06,0.0003,0.3666,30775.9,0.12,0.12,2.212903,2045931.12,0.8,0.38,AAA,0.999,18.75,221.73,176.62
54127,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,42,21.774056,20902.34,455128.72,0.0005,0.2215,86474.5,0.19,0.12,2.891304,2045934.00,0.8,0.33,AAA,0.999,36.82,232.78,189.24
54141,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,42,21.129701,27389.88,578739.97,0.0006,0.1612,98385.8,0.17,0.13,2.989831,2045936.59,0.8,0.31,AAA,0.999,33.36,223.53,181.27
54142,SUCURSAL BARRANQUILLA,DAB14772025,2025.0,42,21.093304,8825.90,186167.39,0.0002,0.4433,31648.5,0.17,0.14,3.664430,2045936.76,1.0,0.46,AAA,0.999,32.17,222.01,179.83


# INVENTARIO ACTUAL

In [8]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [9]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
4,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,PRINCIPAL COTA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL BUCARAGAMNGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [10]:
inv["Cantidad"].sum() 

np.float64(310589.856)

# EVALUACION Y COMPARACION

In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [12]:
df_41 = df_std[df_std["semana"]==41]

df_41["Clasificacion_mia"] = df_41["Clasificacion"]

/tmp/ipykernel_52615/2500382178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_41["Clasificacion_mia"] = df_41["Clasificacion"]


In [13]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_41[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [16]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
17039,SUCURSAL MEDELLIN,DAB08150025,C,C,True,0.00,0.000000,2.48,2,1.93,2
15148,SUCURSAL MEDELLIN,BCS00647125,B,C,False,2.56,0.000000,24.88,0,19.92,0
20236,SUCURSAL NORTE,ALB15W40050,B,A,False,0.34,0.536060,1.30,3,1.11,2
9266,SUCURSAL CALI,DAP08175025,C,C,True,0.00,0.000000,0.00,1,0.00,0
16230,SUCURSAL MEDELLIN,BLS00148125,C,C,True,0.00,0.000000,0.00,0,0.00,0
19715,SUCURSAL MEDELLIN,DCS10353225,C,C,True,0.00,0.000000,0.00,0,0.00,0
22617,SUCURSAL NORTE,DAE06651118,C,C,True,0.00,0.000000,0.00,0,0.00,0
14650,SUCURSAL MEDELLIN,BCE00790125,B,C,False,4.01,0.000000,23.76,0,19.37,0
326,SUCURSAL BARRANQUILLA,BAE03050125,C,C,True,0.00,0.000000,0.00,0,0.00,0
15303,SUCURSAL MEDELLIN,BCS10416125,AAA,AAA,True,25.31,35.692445,138.16,179,113.08,147
